Links to Github:
gh repo clone MicrosoftDocs/SupportArticles-docs : Get Support Troubleshooting
gh repo clone MicrosoftDocs/sql-docs : Get SQL Docs
gh repo clone MicrosoftDocs/azure-databases-docs: Get Azure Databases

In [35]:
# -------------------------
# 1. Imports & Environment
# -------------------------
import os
import re
import json
import pandas as pd
from dotenv import load_dotenv

# For PDF partitioning/extraction
from unstructured.partition.auto import partition
from unstructured.chunking.title import chunk_by_title

# Progress
from tqdm import tqdm
import numpy as np

# For embeddings
import openai
from langchain_openai import AzureOpenAIEmbeddings

# For SQL
import urllib.parse
import pyodbc
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

# For vector store
from langchain_sqlserver import SQLServer_VectorStore
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
import urllib.parse

In [38]:
# Load .env with your environment variables
load_dotenv(override=True)

AZURE_OPENAI_KEY = os.getenv('AZURE_OPENAI_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT_EMBED = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT')
AZURE_OPENAI_DEPLOYMENT = os.getenv('AZURE_OPENAI_DEPLOYMENT')

# SQL DB credentials
# --- Database Configuration ---
server = os.getenv('DB_SERVER')
database = os.getenv('DB_DATABASE')
driver = os.getenv('DB_DRIVER', 'ODBC Driver 18 for SQL Server')
client_id = os.getenv('AZURE_CLIENT_ID')
client_secret = os.getenv('AZURE_CLIENT_SECRET')

# Create SQLAlchemy engine
odbc_str = (
    f"DRIVER={{{driver}}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={client_id};"
    f"PWD={client_secret};"
    "Authentication=ActiveDirectoryServicePrincipal;"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
)

connection_url = f"mssql+pyodbc:///?odbc_connect={quote_plus(odbc_str)}"

engine = create_engine(connection_url, connect_args={"connect_timeout": 30})


# Setup Vector Store
# --- Instantiate your AzureOpenAIEmbeddings ---
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"),
    openai_api_version="2024-10-21",
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
)

# --- Build the VectorStore object ---
vector_store = SQLServer_VectorStore(
    connection_string=connection_url,           # same ODBC DSN used above
    distance_strategy=DistanceStrategy.COSINE,  # or DOT_PRODUCT, etc.
    embedding_function=embeddings,              # text-embedding-ada-002
    embedding_length=1536,                      # Vector dimension
    table_name="DocsChunks_Embeddings",         # Use the name you prefer
)

# Use AzureChatOpenAI for chat completions
llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_deployment=AZURE_OPENAI_DEPLOYMENT,
    openai_api_version="2024-10-21",
    openai_api_key=AZURE_OPENAI_KEY,
)

NameError: name '__file__' is not defined

In [30]:
# ---------------
# 2. PDF Parsing
# ---------------
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 100) -> list:
    """
    Simple text-chunking utility.
    Splits the text into chunks of `chunk_size` characters
    with `overlap` characters overlap between chunks.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap  # move the start back by overlap
        if start < 0:
            start = 0
    return chunks

pdf_path = "RAG_Preparation/SecureBank - Frequently Asked Questions.pdf"

elements = partition(pdf_path)
pdf_text = "\n".join([el.text for el in elements if el.text])  # combine into one big string

# Optional: remove blank lines or do other cleanup
pdf_text = pdf_text.strip()

# Break into smaller chunks for embeddings
chunks = chunk_text(pdf_text, chunk_size=500, overlap=100)


In [31]:
display(chunks)

['SecureBank - Frequently Asked Questions\nThis document provides answers to common questions about SecureBank\'s products and services.\nI. Checking Accounts\nQ: How do I open a new checking account?\nA: You can open a new checking account in one of three ways:\n1. Online: Visit our website at www.securebank.com and click on "Open an Account." The online application takes about 10 minutes to complete. You will need your Social Security Number, a valid government-issued ID, and proof of address.\n2. Mobi',
 'u will need your Social Security Number, a valid government-issued ID, and proof of address.\n2. Mobile App: Download the SecureBank app from the App Store or Google Play. Tap on\n"New User" and follow the prompts to open an account.\n3. In-Person: Visit any of our branch locations during business hours. A personal banker\nwill be happy to assist you with the process.\nQ: What are the benefits of a SecureBank checking account?\nA: Our checking accounts come with a variety of featur

In [32]:
# ----------------------------
# 3. Write raw chunks to SQL
# ----------------------------
# Suppose we create a table [PDF_RawChunks] with columns:
#   id INT IDENTITY(1,1) PRIMARY KEY
#   chunk_text NVARCHAR(MAX)
#   source_pdf NVARCHAR(512)  (optional, if you want to store PDF name/path)
#   created_at DATETIME2 DEFAULT GETDATE() (optional)
#
# Adjust as necessary if your table already exists.

create_table_sql = """
IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[PDF_RawChunks]') AND type in (N'U'))
BEGIN
    CREATE TABLE [dbo].[PDF_RawChunks] (
        [id] INT IDENTITY(1,1) NOT NULL PRIMARY KEY,
        [chunk_text] NVARCHAR(MAX) NOT NULL,
        [source_pdf] NVARCHAR(512) NULL,
        [created_at] DATETIME2 NOT NULL DEFAULT GETDATE()
    );
END
"""
with engine.begin() as conn:
    conn.execute(text(create_table_sql))


test_chunks = chunks[:200]  # comment this out later to process ALL chunks

batch_size = 100

with engine.connect() as conn:
    trans = conn.begin()
    for i, ck in enumerate(
        tqdm(test_chunks, desc="Inserting in 3x100 test batches", unit="chunk"),
        start=1
    ):
        conn.execute(
            text("INSERT INTO [dbo].[PDF_RawChunks] (chunk_text, source_pdf) VALUES (:ctext, :spdf)"),
            {"ctext": ck, "spdf": pdf_path}
        )

        # Commit every 100 inserts
        if i % batch_size == 0:
            trans.commit()
            # If there's more data left to insert, start a new transaction
            if i < len(test_chunks):
                trans = conn.begin()

    # If the total number isn't an exact multiple of batch_size
    # commit leftover rows in the final partial batch
    if len(test_chunks) % batch_size != 0:
        trans.commit()

print(f"Inserted {len(test_chunks)} chunks in batches of {batch_size}.")

Inserting in 3x100 test batches: 100%|██████████| 23/23 [00:04<00:00,  5.31chunk/s]



Inserted 23 chunks in batches of 100.


In [33]:
# 1) Fetch Chunks from PDF_RawChunks That Need Embeddings
select_sql = """
SELECT RC.id, RC.chunk_text, RC.source_pdf
FROM PDF_RawChunks RC
WHERE NOT EXISTS (
    SELECT 1
    FROM DocsChunks_Embeddings VEC
    WHERE VEC.custom_id = CAST(RC.id AS VARCHAR(50))
)
ORDER BY RC.id
"""

with engine.connect() as conn:
    result = conn.execute(text(select_sql))
    rows = result.fetchall()

print(f"Found {len(rows)} row(s) in PDF_RawChunks with no existing embedding.")

# 2) Insert in Batches via vector_store.add_texts()

batch_size = 100  # commit in batches of 100

# Convert each row into text + metadata
all_texts = []
all_metadata = []

for row in rows:
    row_id   = row[0]
    text_val = row[1]
    pdf_path = row[2]

    # build your metadata
    meta_dict = {
        "custom_id": str(row_id),   # store the PDF_RawChunks ID as a string
        "source_pdf": pdf_path
    }

    all_texts.append(text_val)
    all_metadata.append(meta_dict)

print(f"Preparing to insert {len(all_texts)} texts in batches of {batch_size}...")

from math import ceil

num_rows = len(all_texts)
num_batches = ceil(num_rows / batch_size)

index = 0
for b in range(num_batches):
    # Slice out a batch
    batch_texts = all_texts[index : index+batch_size]
    batch_meta  = all_metadata[index : index+batch_size]
    index += batch_size

    print(f"Batch {b+1}/{num_batches}: inserting {len(batch_texts)} items...")
    
    # The vector_store call *immediately* does embeddings + inserts
    # into the underlying table. So each call is effectively a "mini commit."
    vector_store.add_texts(
        texts=batch_texts,
        metadatas=batch_meta
    )

print("All missing rows have been embedded and inserted into PDF_RawChunks_Embeddings!")

Found 46 row(s) in PDF_RawChunks with no existing embedding.
Preparing to insert 46 texts in batches of 100...
Batch 1/1: inserting 46 items...
All missing rows have been embedded and inserted into PDF_RawChunks_Embeddings!
All missing rows have been embedded and inserted into PDF_RawChunks_Embeddings!


In [34]:
query = "What are the late payment fees on credit cards?"
docs = vector_store.similarity_search(query, k=3)
for i, doc in enumerate(docs, 1):
    print(f"\nResult {i}:\nMetadata = {doc.metadata}\nText    = {doc.page_content[:150]}...")


Result 1:
Metadata = {'custom_id': '11', 'source_pdf': 'RAG_Preparation/SecureBank - Frequently Asked Questions.pdf'}
Text    = te, the ranges are:
Cash Back Rewards Card: 15.99% - 23.99% ● Travel Rewards Card: 16.99% - 24.99% ● Secured Credit Card: Fixed at 21.99% ● Low APR Ca...

Result 2:
Metadata = {'custom_id': '34', 'source_pdf': 'RAG_Preparation/SecureBank - Frequently Asked Questions.pdf'}
Text    = te, the ranges are:
Cash Back Rewards Card: 15.99% - 23.99% ● Travel Rewards Card: 16.99% - 24.99% ● Secured Credit Card: Fixed at 21.99% ● Low APR Ca...

Result 3:
Metadata = {'custom_id': '12', 'source_pdf': 'RAG_Preparation/SecureBank - Frequently Asked Questions.pdf'}
Text    = yments may be reported to credit bureaus, which can negatively impact your credit score. If you are having trouble making a payment, please contact us...
